# Coleta de Dados com o Selenium
    Coletar dados de 250 séries com as maiores avaliações do site `https://www.imdb.com/pt/

In [ ]:
import logging
import re
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from concurrent.futures import ThreadPoolExecutor, as_completed
import json
from dataclasses import asdict

# Configuração dos logs
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    datefmt="%H:%M:%S"
)

def scrape_serie(serie_data):
    """Abre um navegador separado e coleta dados de uma série"""
    driver = webdriver.Chrome()
    wait = WebDriverWait(driver, 10)

    try:
        driver.get(serie_data["link"])

        popularidade = wait.until(
            EC.presence_of_element_located((
                By.XPATH,
                '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[2]/div/div[3]/a/span/div/div[2]/div[1]'
            ))
        ).text

        elenco_section = wait.until(
            EC.presence_of_all_elements_located((
                By.XPATH,
                '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[5]'
            ))
        )
        elenco_principal = [ator.text for ator in elenco_section]

        return {
            "info": serie_data["info"],
            "link": serie_data["link"],
            "popularidade": popularidade,
            "elenco_principal": elenco_principal
        }

    except Exception as e:
        logging.error(f"Erro em {serie_data['link']}: {e}")
        return None
    finally:
        driver.quit()


# ----------------------
# COLETA DOS LINKS
# ----------------------
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)

driver.get("https://www.imdb.com/pt/")

menu_icon = wait.until(
    EC.element_to_be_clickable((By.XPATH, '//*[@id="imdbHeader-navDrawerOpen"]'))
)
menu_icon.click()

link_series_bem_avaliadas = wait.until(
    EC.element_to_be_clickable((
        By.XPATH,
        '//*[@id="imdbHeader"]/div/aside[1]/div/div[2]/div/div[2]/div[1]/span/div/div/ul/a[2]'
    ))
)
link_series_bem_avaliadas.click()

series = wait.until(
    EC.presence_of_all_elements_located((
        By.XPATH,
        '//*[@id="__next"]/main/div/div[3]/section/div/div[2]/div/ul/li'
    ))
)

series_data = []
for serie in series:
    try:
        info = serie.text.split("\n")
        link = serie.find_element(By.TAG_NAME, 'a').get_attribute('href')
        series_data.append({"info": info, "link": link})
    except Exception as e:
        logging.warning(f"Erro ao extrair série: {e}")

driver.quit()

# ----------------------
# COLETA EM PARALELO
# ----------------------
logging.info(f"Encontradas {len(series_data)} séries. Iniciando coleta em paralelo...")

start_time = time.time()
dados_completos = []
total = len(series_data)
coletadas = 0

with ThreadPoolExecutor(max_workers=5) as executor:  # até 5 navegadores em paralelo
    futures = [executor.submit(scrape_serie, s) for s in series_data]
    for future in as_completed(futures):
        result = future.result()
        coletadas += 1
        if result:
            dados_completos.append(result)
            logging.info(
                f"Coletada: {result['info'][0]} "
                f"({coletadas}/{total} - {coletadas/total*100:.1f}%)"
            )
        else:
            logging.warning(f"Série falhou ({coletadas}/{total} - {coletadas/total*100:.1f}%)")

end_time = time.time()
logging.info(f"Coleta finalizada em {end_time - start_time:.2f} segundos.")

print(dados_completos)


20:46:17 [INFO] Encontradas 175 séries. Iniciando coleta em paralelo...
20:46:27 [INFO] Coletada: 5. Chernobyl (1/175 - 0.6%)
20:46:27 [INFO] Coletada: 3. Planeta Terra (2/175 - 1.1%)
20:46:27 [INFO] Coletada: 2. Planeta Terra II (3/175 - 1.7%)
20:46:31 [INFO] Coletada: 4. Irmãos de Guerra (4/175 - 2.3%)
20:46:32 [INFO] Coletada: 1. Breaking Bad (5/175 - 2.9%)
20:46:35 [INFO] Coletada: 6. A Escuta (6/175 - 3.4%)
20:46:36 [INFO] Coletada: 7. Avatar: A Lenda de Aang (7/175 - 4.0%)
20:46:36 [INFO] Coletada: 8. Família Soprano (8/175 - 4.6%)
20:46:37 [INFO] Coletada: 10. Cosmos: Uma Odisseia do Espaço-Tempo (9/175 - 5.1%)
20:46:39 [INFO] Coletada: 9. Planeta Azul II (10/175 - 5.7%)
20:46:44 [INFO] Coletada: 12. Nosso Planeta (11/175 - 6.3%)
20:46:45 [INFO] Coletada: 13. Game of Thrones (12/175 - 6.9%)
20:46:49 [INFO] Coletada: 15. O Mundo em Guerra (13/175 - 7.4%)
20:46:49 [INFO] Coletada: 14. Bluey (14/175 - 8.0%)
20:46:49 [INFO] Coletada: 11. Cosmos (15/175 - 8.6%)
20:46:50 [INFO] Coleta

[{'info': ['5. Chernobyl', '20195 episódios16Minissérie de televisão', '9,3', ' (978 mil)', 'Avaliar', 'Marcar como assistido'], 'link': 'https://www.imdb.com/pt/title/tt7366338/?ref_=chttvtp_i_5', 'popularidade': '119', 'elenco_principal': ["Elenco principal\n97\nEditar\nJessie Buckley\nLyudmilla Ignatenko\n5 episódios • 2019\nJared Harris\nValery Legasov\n5 episódios • 2019\nStellan Skarsgård\nBoris Shcherbina\n5 episódios • 2019\nAdam Nagaitis\nVasily Ignatenko\n4 episódios • 2019\nEmily Watson\nUlana Khomyuk\n4 episódios • 2019\nPaul Ritter\nAnatoly Dyatlov\n4 episódios • 2019\nRobert Emms\nLeonid Toptunov\n4 episódios • 2019\nSam Troughton\nAlexandr Akimov\n4 episódios • 2019\nKarl Davies\nViktor Proskuryakov\n3 episódios • 2019\nMichael Socha\nMikhail\n3 episódios • 2019\nLaura Elphinstone\nOksana\n3 episódios • 2019\nJan Ricica\nOksana's Kid\n3 episódios • 2019\nAdrian Rawlins\nNikolai Fomin\n3 episódios • 2019\nAlan Williams\nKGB Chairman Charkov\n3 episódios • 2019\nCon O'Neil

# Teste com BeautifulSoup

In [ ]:
import asyncio
import aiohttp
import logging
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s", datefmt="%H:%M:%S")

# ---------------------- PEGAR LINKS COM SELENIUM ----------------------
def coletar_links():
    driver = webdriver.Chrome()
    wait = WebDriverWait(driver, 10)

    driver.get("https://www.imdb.com/pt/")

    menu_icon = wait.until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="imdbHeader-navDrawerOpen"]'))
    )
    menu_icon.click()

    link_series_bem_avaliadas = wait.until(
        EC.element_to_be_clickable((
            By.XPATH,
            '//*[@id="imdbHeader"]/div/aside[1]/div/div[2]/div/div[2]/div[1]/span/div/div/ul/a[2]'
        ))
    )
    link_series_bem_avaliadas.click()

    series = wait.until(
        EC.presence_of_all_elements_located((
            By.XPATH,
            '//*[@id="__next"]/main/div/div[3]/section/div/div[2]/div/ul/li'
        ))
    )

    series_data = []
    for serie in series:
        try:
            info = serie.text.split("\n")
            link = serie.find_element(By.TAG_NAME, 'a').get_attribute('href')
            series_data.append({"info": info, "link": link})
        except Exception as e:
            logging.warning(f"Erro ao extrair série: {e}")

    driver.quit()
    return series_data

# ---------------------- FUNÇÃO ASSÍNCRONA DE COLETA ----------------------
async def fetch_and_parse(session, serie_data, idx, total):
    try:
        async with session.get(serie_data["link"]) as response:
            html = await response.text()
            soup = BeautifulSoup(html, "html.parser")

            # Popularidade (tentando pegar o equivalente do XPath usado no Selenium)
            popularidade_tag = soup.select_one('a[href*="ratings"] span')
            popularidade = popularidade_tag.get_text(strip=True) if popularidade_tag else "N/A"

            # Elenco principal
            elenco_section = soup.select("section[data-testid='title-cast'] li.ipc-inline-list__item")
            elenco_principal = [ator.get_text(" ", strip=True) for ator in elenco_section]

            logging.info(f"Coletada {idx}/{total} - {serie_data['info'][0]}")

            return {
                "info": serie_data["info"],
                "link": serie_data["link"],
                "popularidade": popularidade,
                "elenco_principal": elenco_principal
            }
    except Exception as e:
        logging.error(f"Erro em {serie_data['link']}: {e}")
        return None

async def coletar_detalhes(series_data):
    start_time = time.time()
    total = len(series_data)

    async with aiohttp.ClientSession() as session:
        tasks = [fetch_and_parse(session, s, idx + 1, total) for idx, s in enumerate(series_data)]
        results = await asyncio.gather(*tasks)

    elapsed = time.time() - start_time
    logging.info(f"Coleta finalizada em {elapsed:.2f} segundos.")
    return [r for r in results if r]

# ---------------------- EXECUÇÃO NO JUPYTER ----------------------
series_data = coletar_links()
logging.info(f"{len(series_data)} links coletados. Iniciando coleta assíncrona...")

# No Jupyter, usar await
dados_completos = await coletar_detalhes(series_data)

# Resultado final
print(dados_completos)


ModuleNotFoundError: No module named 'aiohttp'

### Fase II Tratamento de dados
    - Criar funções que possibilitam tratar as listas de dados
    - Criar objetos utilizando essas listas
    - Criar um arquivo Json

In [ ]:
from dataclasses import dataclass

@dataclass
class Serie:
    nome: str
    ano_inicio: int
    episodios: int
    faixa_etaria: int  # Pode ser número ou "Livre"
    avaliacao: float
    link: str
    popularidade: float
    atores: list

    def to_dict(self):
        # Transforma o objeto em Dicionario
        return {
            "Titulo": self.nome,
            "Ano de Estreia": self.ano_inicio,
            "Episodios": self.episodios,
            "Classificação Indicativa": self.faixa_etaria,
            "Nota do IMDB": self.avaliacao,
            "Link": self.link,
            "Popularidade": self.popularidade,
            "Elenco Principal": self.atores}



def tratar_string(string):
    # Normaliza traço en dash
    string = string.replace("–", "-")
    # Extrai anos
    anos = re.findall(r"\d{4}", string)
    # Inicializa
    episodios = None
    faixa = None
    # Regex: pega número de episódios e a faixa etária
    match = re.search(r"(\d+)\s*episódios\s*(\S+)?", string, re.IGNORECASE)
    if match:
        full_number = match.group(1)
        episodios = int(full_number[-2:])
        faixa_raw = match.group(2)

        if faixa_raw:
            numeros = re.findall(r"\d+", faixa_raw)  # pega todos os números
            if numeros:
                faixa = int(numeros[0])  # converte o primeiro número que encontrar
            else:
                faixa = 0    # se não tiver número, pega os 5 primeiros caracteres

    return anos, episodios, faixa


def tratar_elenco(texto):
    resultados = []
    if isinstance(texto, list):
        texto = "\n".join(texto)
    padrao = re.compile(
        r"([^\n]+)\n"                    # Ator
        r"([^\n]+)\n"                    # Personagem 
        r"(\d+)\s*episódio[s]?",         # Episódios
        re.IGNORECASE)

    for ator, personagem, ep in padrao.findall(texto):
        resultados.append([ator.strip(), personagem.strip(), int(ep)])
    return resultados


def tratamento_geral(dicionario:dict):
    # --Função para o tratamento geral dos dados-- #
    for key, item in dicionario.items():
        # Trata a primeira parte dos dados a 'info' que contem (nome, ano, ep, fiaxa e avaliação)
        if key == 'info':
            nome = item[0]
            avaliacao = item[2]
            avaliacao = float(avaliacao.replace(",", "."))
            ano, episodio, faixa = tratar_string(item[1])
            faixa = int(faixa)
            if len(ano) > 1:
                ano = "-".join(ano)
            else:
                ano = int(ano[0])
            
        # Trata o link
        elif key == 'link':
            link = item
        # Trata a popularidade
        elif key == 'popularidade':
            pop = float(item)
        # Trata a lista de Atores
        elif key == 'elenco_principal':
            lista_de_atores = tratar_elenco(item)

    return Serie(nome, ano, episodio, faixa, avaliacao, link, pop, lista_de_atores)

def covert_json(lista):
    lista_dicts = [s.to_dict() for s in lista]
    with open("series.json", "w", encoding="utf-8") as f:
        json.dump(lista_dicts, f, ensure_ascii=False, indent=4)


lista_de_obj = list(map(tratamento_geral, dados_completos))
covert_json(lista_de_obj)

print(lista_de_obj)
print(dados_completos[159])


[Serie(nome='5. Chernobyl', ano_inicio=2019, episodios=95, faixa_etaria=16, avaliacao=9.3, link='https://www.imdb.com/pt/title/tt7366338/?ref_=chttvtp_i_5', popularidade=119.0, atores=[['Jessie Buckley', 'Lyudmilla Ignatenko', 5], ['Jared Harris', 'Valery Legasov', 5], ['Stellan Skarsgård', 'Boris Shcherbina', 5], ['Adam Nagaitis', 'Vasily Ignatenko', 4], ['Emily Watson', 'Ulana Khomyuk', 4], ['Paul Ritter', 'Anatoly Dyatlov', 4], ['Robert Emms', 'Leonid Toptunov', 4], ['Sam Troughton', 'Alexandr Akimov', 4], ['Karl Davies', 'Viktor Proskuryakov', 3], ['Michael Socha', 'Mikhail', 3], ['Laura Elphinstone', 'Oksana', 3], ['Jan Ricica', "Oksana's Kid", 3], ['Adrian Rawlins', 'Nikolai Fomin', 3], ['Alan Williams', 'KGB Chairman Charkov', 3], ["Con O'Neill", 'Viktor Bryukhanov', 3], ['Douggie McMeekin', 'Yuvchenko', 2], ['Nadia Clifford', 'Dr. Svetlana Zinchenko', 2], ['David Dencik', 'Mikhail Gorbachev', 2]]), Serie(nome='3. Planeta Terra', ano_inicio=2006, episodios=11, faixa_etaria=0, av